In [1]:
# import
import sys
import torch
import numpy as np
import pandas as pd
import random
import nltk
import nltk
from nltk.corpus import names
from nltk.corpus import wordnet as wn
import os
from typing import Dict
import torch
from datasets import load_dataset, Dataset, DatasetDict
import datasets
from collections import Counter
import inflect
from fuzzywuzzy import process

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('names')
print("WordNet {}".format(wn.get_version())) # print WordNet version

[nltk_data] Downloading package wordnet to C:\Users\Mattia
[nltk_data]     Maffo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Mattia
[nltk_data]     Maffo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package names to C:\Users\Mattia
[nltk_data]     Maffo\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


WordNet 3.0


In [4]:
# load the dataset
dataset = load_dataset("tommasobonomo/sem_augmented_fever_nli")

# Let's see an example...
print(dataset)
print(dataset['train'][0])
print("Label of the first sentence:", dataset['train'][0]['label']) 

# Take a look at the labels
all_labels = []
for label in dataset['train'].unique("label"):
    all_labels.append(label)

print(all_labels) ## ['ENTAILMENT', 'NEUTRAL', 'CONTRADICTION'] == ['SUPPORTS', 'NOT ENOUGH INFO', 'REFUTES']

# Label mapping

label_map = {
    'ENTAILMENT': 0,
    'NEUTRAL': 1,
    'CONTRADICTION': 2
}

def convert_labels(example):
    example['label'] = label_map[example['label']]
    return example

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 51086
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2288
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2287
    })
})
{'id': '150448', 'premise': "Roman Atwood . He is best known for his vlogs , where he posts updates about his life on a daily basis . His vlogging channel , `` RomanAtwoodVlogs '' , has a total of 3.3 billion views and 11.9 million subscribers . He also has another YouTube channel called `` RomanAtwood '' , where he posts pranks .", 'hypothesis': 'Roman Atwood is a content creator.', 'label': 'ENTAILMENT', 'wsd': {'premise': [{'index': 0, 'text': 'Roman', 'pos': 'ADJ', 'lemma': 'roman', 'bnSynsetId': 'bn:00109913a', 'wnSynsetOffset': '2921569a', 'nltkSynset': 'roman.a.01'}, {'index': 1, 'text': '

In [5]:
train_dataset = dataset['train'].map(convert_labels)
val_dataset = dataset['validation'].map(convert_labels)
test_dataset = dataset['test'].map(convert_labels)


In [6]:
# Now look at the WSD and SRL examples...
print("WSD of the first sentence:", dataset['train'][0]['wsd'])
print("SRl of the first sentence:", dataset['train'][0]['srl'])

# What's the len of the dataset?
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(val_dataset))
print("Number of test examples:", len(test_dataset))

WSD of the first sentence: {'premise': [{'index': 0, 'text': 'Roman', 'pos': 'ADJ', 'lemma': 'roman', 'bnSynsetId': 'bn:00109913a', 'wnSynsetOffset': '2921569a', 'nltkSynset': 'roman.a.01'}, {'index': 1, 'text': 'Atwood', 'pos': 'PROPN', 'lemma': 'Atwood', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 2, 'text': '.', 'pos': 'PUNCT', 'lemma': '.', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 3, 'text': 'He', 'pos': 'PRON', 'lemma': 'he', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 4, 'text': 'is', 'pos': 'AUX', 'lemma': 'be', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 5, 'text': 'best', 'pos': 'ADV', 'lemma': 'well', 'bnSynsetId': 'bn:00117603r', 'wnSynsetOffset': '12779r', 'nltkSynset': 'well.r.02'}, {'index': 6, 'text': 'known', 'pos': 'VERB', 'lemma': 'know', 'bnSynsetId': 'bn:00090143v', 'wnSynsetOffset': '594337v', 'nltkSynset': 'know.v.04'}, {'index': 7, 'text': 'for', 'po

In [7]:
# let's count the labels
label_counts = Counter(train_dataset['label'])
print("Label distribution in the training set:", label_counts)

Label distribution in the training set: Counter({0: 31128, 2: 12331, 1: 7627})


I plan to develop an automated pipeline for generating an adversarial training set using the attributes 'wsd' (word sense disambiguation) and 'srl' (semantic role labeling) from my dataset. The goal is to modify one or more words in each sentence to increase its complexity while keeping it semantically equivalent and maintaining the same final label (entailment, contradictory, or neutral) or changing with the opposite.

To achieve this, I will apply various techniques to different subsets of the provided training set. By diversifying the techniques used across different subsets, I aim to enhance the model's robustness and ability to handle more complex and variable inputs.

In [8]:
# USED FOR DEBUGGING

subset_entailment = train_dataset.filter(lambda example: example['label'] == 0).select(range(200))
print("Number of samples in the entailment subset:", len(subset_entailment))
for sample in subset_entailment:
    print(sample['hypothesis'])

Number of samples in the entailment subset: 200
Roman Atwood is a content creator.
The Boston Celtics play their home games at TD Garden.
There is a movie called The Hunger Games.
Ryan Seacrest is a person.
Stranger than Fiction is a film.
Selena recorded music.
Selena recorded music.
Selena recorded music.
Selena recorded music.
Tupac Shakur was born Lesane Parish Crooks.
Lisbon has a population larger than 1.
Newfoundland and Labrador is the most linguistically homogeneous of Canada.
Sophie Turner was born in the 1990s.
The Smurfs (film) was released.
Hacksaw Ridge came out in America on November 4, 2016.
Michael Fassbender is an actor.
Michael Fassbender is an actor.
Uzbekistan is a part of the Commonwealth of Independent States (CIS).
Brian Wilson was part of the Beach Boys.
Led Zeppelin is a band.
Led Zeppelin is a band.
Richard Harris was born on October 25, 2002.
The Fosters is a television series.
Liverpool F.C. was valued at $1.55 billion at one point.
Land Rover makes cars.
O

In [9]:
# USED FOR DEBUGGING

example = subset_entailment[0]['hypothesis']
print("Example hypothesis:", example)
splitted_example = example.split()
print("Example hypothesis splitted:", splitted_example)

wsd = subset_entailment[0]['wsd']['hypothesis']
print("Example WSD:", wsd)

srl1 = srl = subset_entailment[0]['srl']['hypothesis']
print("Example SRL1:", srl1)
srl = subset_entailment[0]['srl']['hypothesis']['annotations'][0]['verbatlas']
print("Example SRL:", srl)

for sample in subset_entailment:
    annotations = sample['srl']['hypothesis']['annotations']
    for annotation in annotations:
        print("Annotation:", annotation)
        token_idx = annotation['tokenIndex']
        print("Token index:", token_idx)
        for role in annotation['verbatlas']['roles']:
            role_name = role['role']
            print("Role:", role_name)
            if role_name == 'Agent':
                print("Role_name:", role_name)
                break
            

Example hypothesis: Roman Atwood is a content creator.
Example hypothesis splitted: ['Roman', 'Atwood', 'is', 'a', 'content', 'creator.']
Example WSD: [{'index': 0, 'text': 'Roman', 'pos': 'PROPN', 'lemma': 'Roman', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 1, 'text': 'Atwood', 'pos': 'PROPN', 'lemma': 'Atwood', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 2, 'text': 'is', 'pos': 'AUX', 'lemma': 'be', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 3, 'text': 'a', 'pos': 'DET', 'lemma': 'a', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 4, 'text': 'content', 'pos': 'ADJ', 'lemma': 'content', 'bnSynsetId': 'bn:00100364a', 'wnSynsetOffset': '588797a', 'nltkSynset': 'contented.a.01'}, {'index': 5, 'text': 'creator', 'pos': 'NOUN', 'lemma': 'creator', 'bnSynsetId': 'bn:00023660n', 'wnSynsetOffset': '9614315n', 'nltkSynset': 'creator.n.02'}, {'index': 6, 'text': '.', 'pos': 'PUNCT', 'l

In [10]:
import spacy
nlp = spacy.load('en_core_web_md')
random.seed(42)

In [11]:
# Now, let's check if the two sentences are similar.

def are_sentences_similar(sentence1, sentence2, threshold=0.7):
    doc1 = nlp(sentence1)
    doc2 = nlp(sentence2)
    
    similarity = doc1.similarity(doc2)
    print("Similarity score:", similarity)
    return similarity >= threshold

In [11]:
# Library for checking if a word is plural or singular
p = inflect.engine()

In [12]:
from transformers import BertTokenizer, BertForMaskedLM
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

# Bert model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_1 = BertForMaskedLM.from_pretrained('bert-base-uncased')
embedder_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
embedder_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


# Function to get the synset of a word
def get_synset(word, hypothesis):
    for word_info in hypothesis:
        if word_info['text'] == word:
            if word_info['nltkSynset'] != 'O':
                return wn.synset(word_info['nltkSynset'])
            else:
                return None
    return None

# Function to get the hypernyms and hyponyms of a synset
def get_hypernyms(synset):
    hypernyms = set()
    for hypernym in synset.hypernyms():
        hypernyms.add(hypernym)
    return list(hypernyms)

def get_hyponyms(synset):
    hyponyms = set()
    for hyponym in synset.hyponyms():
        hyponyms.add(hyponym)
    return list(hyponyms)

# Function to get the embedding of a text
def get_embedding(text):
    inputs = embedder_tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = embedder_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach()

# FIRST TECHNIQUE: Replace a word with a hypernym or hyponym w.r.t. the prediction of the BERT model
def replace_with_hypernym_hyponym(sample, target_word):
    sentence = sample.copy()
    synset = get_synset(target_word, sentence['wsd']['hypothesis'])
    if not synset:
        return sentence
    
    hypernyms = get_hypernyms(synset)
    hyponyms = get_hyponyms(synset)
    
    candidates = set()
    for hypernym in hypernyms:
        candidates.update(hypernym.lemmas())
    for hyponym in hyponyms:
        candidates.update(hyponym.lemmas())
    
    if not candidates:
        return sentence
    
    candidate_lemmas = [lemma.name() for lemma in candidates]
    
    # Tokenize the sentence
    tokens = nltk.word_tokenize(sentence['hypothesis'])
    
    # Find the index of the target word
    try:
        target_index = tokens.index(target_word)
    except ValueError:
        return sentence

    # Replace the target word with [MASK]
    tokens[target_index] = '[MASK]'
    masked_sentence = ' '.join(tokens)
    print(masked_sentence)
    
    # Tokenize now the sentence with the mask
    inputs = tokenizer(masked_sentence, return_tensors='pt')
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    print(mask_token_index)

    # Get the output of the model
    with torch.no_grad():
        outputs = model_1(**inputs)
        logits = outputs.logits
    
    mask_token_logits = logits[0, mask_token_index, :]

    # Get the top 100 predicted tokens
    top_tokens = torch.topk(mask_token_logits, 100, dim=1).indices[0].tolist()
    predicted_tokens = tokenizer.convert_ids_to_tokens(top_tokens)

    # Compute the similarity between the predicted tokens and the candidate lemmas
    candidate_embeddings = {lemma: get_embedding(lemma).squeeze(0) for lemma in candidate_lemmas}
    similarities = []

    for token in predicted_tokens:
        candidate_word = token.strip()
        candidate_embedding = get_embedding(candidate_word).squeeze(0)
        for lemma, lemma_embedding in candidate_embeddings.items():
            similarity = cosine_similarity(lemma_embedding.unsqueeze(0), candidate_embedding.unsqueeze(0))[0][0]
            similarities.append((candidate_word, lemma, similarity))
    
    # Find the best candidate
    best_candidate = max(similarities, key=lambda x: x[2])
    best_word = best_candidate[0]
    
    if best_candidate[2] > 0.5:  # Threshold for the similarity
        singular_form = p.singular_noun(target_word)  # Transform the word to plural if the target word is plural
        if singular_form is not False:
            best_word = p.plural(best_candidate[0])
        tokens[target_index] = best_word
        new_sentence = ' '.join(tokens)
        sentence['hypothesis'] = new_sentence
        return sentence
    
    # Else return the original sentence
    return sentence

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
from sentence_transformers import SentenceTransformer, util
model_2 = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def calculate_similarity(original_sentence, swapped_sentence):
    # Find the embeddings
    original_embedding = model_2.encode(original_sentence, convert_to_tensor=True)
    swapped_embedding = model_2.encode(swapped_sentence, convert_to_tensor=True)
    
    # Compute the similarity
    similarity_score = util.pytorch_cos_sim(original_embedding, swapped_embedding)
    return similarity_score.item()

def check_and_update_label_dataset(original_sentence, swapped_sentence, threshold=0.989):
    o_sentence = original_sentence['hypothesis']
    
    similarity = calculate_similarity(o_sentence, swapped_sentence)
    
    # If the similarity is below the threshold, change the label with the opposite.
    label = 0
    if similarity < threshold:
        if original_sentence['label'] == 0:
            label = 2
        elif original_sentence['label'] == 2:
            label = 0
        else:
            label = 1
        print("Updated label:", label)
    
    return swapped_sentence, label


c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
# SECOND TECHNIQUE: Swap the agent and patient roles in the SRL annotation of the hypothesis

def swap_agent_patient(srl):
    srl_swapped = srl.copy()
    annotations = srl_swapped['annotations']
    trovato = 0
    for annotation in annotations:
        roles = annotation['verbatlas']['roles']
        agent_index, patient_index = None, None
        for idx, role in enumerate(roles):
            if role['role'] == 'Agent':
                agent_index = idx
            elif role['role'] == 'Patient':
                patient_index = idx
        if agent_index is not None and patient_index is not None:
            # Swap the spans
            roles[agent_index]['span'], roles[patient_index]['span'] = roles[patient_index]['span'], roles[agent_index]['span']
            # Swap the tokens
            tokens = srl_swapped['tokens']
            tokens[roles[agent_index]['span'][0]:roles[agent_index]['span'][1]], tokens[roles[patient_index]['span'][0]:roles[patient_index]['span'][1]] = tokens[roles[patient_index]['span'][0]:roles[patient_index]['span'][1]], tokens[roles[agent_index]['span'][0]:roles[agent_index]['span'][1]]
            srl_swapped['tokens'] = tokens
            trovato = 1
    return srl_swapped, trovato

# Apply the swap
def apply_swap(sample):
    sentence = sample.copy()
    swapped_sample = []
    srl_swapped, trovato = swap_agent_patient(sentence['srl']['hypothesis'])
    new_hypothesis = ' '.join([token['rawText'] for token in srl_swapped['tokens']])
    label = sentence['label']
    hypothesis_with_changed_label = new_hypothesis
    if trovato == 1:
        print("Original hypothesis:", sentence['hypothesis'])
        # Turn the first letter to uppercase
        new_hypothesis = new_hypothesis[0].upper() + new_hypothesis[1:]
        hypothesis_with_changed_label, label = check_and_update_label_dataset(sentence, new_hypothesis)
        print("Swapped hypothesis:", hypothesis_with_changed_label)
    swapped_sample.append({
        'id': sentence['id'],
        'premise': sentence['premise'],
        'hypothesis': hypothesis_with_changed_label,
        'label': label,
        'wsd': sentence['wsd'],
        'srl': srl_swapped
    })
    return swapped_sample

In [16]:
# THIRD TECHNIQUE: Replace a word with its antonym

# function to get antonyms of a word
def get_antonym(synset):
    lemma = synset.lemmas()[0]
    return lemma.antonyms()[0].name() if lemma.antonyms() else []

nlp = spacy.load("en_core_web_sm")

# THIRD TECHNIQUE: Replace a word with its antonym
def replace_with_antonym(sample):
    sentence = sample.copy()
    words = [word_info['text'] for word_info in sentence['wsd']['hypothesis'] if word_info['pos'] in ['ADJ', 'ADV']]
    print("Words:", words)
    words_to_replace = random.sample(words, min(1, len(words)))
    for word in words_to_replace:
        doc = nlp(sentence['hypothesis'])
        comparatives = [token.text for token in doc if token.tag_ == "JJR"]
        print("Comparatives:", comparatives)
        if word in comparatives:
            words_to_replace.remove(word)
            words_to_replace = random.sample(words, min(1, len(words)))
            if words_to_replace[0] == word:
                return sentence
        synset = get_synset(word, sentence['wsd']['hypothesis'])
        if not synset:
            return sentence
        antonym = get_antonym(synset)
        print("Antonym of {}: {}".format(word, antonym))
        if not antonym:
            return sentence
        # substitute the word in the sentence accessing the sentence['hypothesis'] and replacing the word with its antonym
        sentence['hypothesis'] = sentence['hypothesis'].replace(word, antonym)
        # transform the first letter to uppercase
        sentence['hypothesis'] = sentence['hypothesis'][0].upper() + sentence['hypothesis'][1:]
        if sentence['label'] == 0:
            sentence['label'] = 2
        elif sentence['label'] == 2:
            sentence['label'] = 0
        else:
            sentence['label'] = 1
    return sentence
    

Words: ['content']
Comparatives: []
Antonym of content: discontented
Words: []
Words: []
Words: []
Words: ['Stranger']
Comparatives: ['Stranger']
Words: []
Words: []
Words: []
Words: []
Words: []
Words: ['larger']
Comparatives: ['larger']
Words: ['most', 'linguistically', 'homogeneous']
Comparatives: []
Antonym of most: least
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: ['cosmetic']
Comparatives: []
Antonym of cosmetic: []
Words: []
Words: []
Words: ['Canadian']
Comparatives: []
Antonym of Canadian: []
Words: []
Words: []
Words: []
Words: ['underwent']
Comparatives: []
Antonym of underwent: []
Words: []
Words: []
Words: []
Words: []
Words: []
Words: ['teen']
Comparatives: []
Antonym of teen: []
Words: []
Words: ['precise']
Comparatives: []
Antonym of precise: imprecise
Words: []
Words: []
Words: []
Words: ['cultural']
Comparatives: []
Antonym of cultural: 

In [17]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load the tokenizer and model
model_name = "tuner007/pegasus_paraphrase"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model_p = PegasusForConditionalGeneration.from_pretrained(model_name)

# FOURTH TECHNIQUE: Paraphrase the sentence using Pegasus

def paraphrase(sample, num_return_sequences=1, num_beams=5):
    sentence = sample.copy()
    # Tokenize the input sentence
    inputs = tokenizer([sentence['hypothesis']], truncation=True, padding="longest", max_length=60, return_tensors="pt")
    
    # Generate paraphrases
    outputs = model_p.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=60,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        temperature=1.5
    )

    # Decode the outputs
    paraphrased_sentences = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return paraphrased_sentences


# Now check which paraphrases are similar to the original sentence, computing the similarity score and then choose randomly one of them. If the score is below a certain threshold, change the label to 'contradiction' and viceversa.
def check_paraphrases(sample, paraphrases):
    sentence = sample.copy()
    original_sentence = sentence['hypothesis']
    similarity_scores = []
    for paraphrase in paraphrases:
        similarity = calculate_similarity(original_sentence, paraphrase)
        similarity_scores.append(similarity)
    
    print("Similarity scores:", similarity_scores)
    max_similarity = max(similarity_scores)
    best_paraphrase = paraphrases[similarity_scores.index(max_similarity)]
    
    return best_paraphrase

# Example usage
# for sentence_data in subset_entailment:
#     paraphrased_sentences = paraphrase(sentence_data, num_return_sequences=3)
#     for i, paraphrased_sentence in enumerate(paraphrased_sentences, 1):
#         print(f"Paraphrase {i}: {paraphrased_sentence}")
#     best_paraphrase = check_paraphrases(sentence_data, paraphrased_sentences)
#     print("Best paraphrase:", best_paraphrase)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
male_names = names.words('male.txt')
female_names = names.words('female.txt')

print(male_names[:10])
print(female_names[:10])

url = "https://raw.githubusercontent.com/datasets/world-cities/master/data/world-cities.csv"
cities_df = pd.read_csv(url)
cities = cities_df['name'].tolist()

print(cities[:10])

def generate_replacement_dict():
    replacement_dict = {}
    for sample in subset_entailment:
        words = [word_info['text'] for word_info in sample['wsd']['hypothesis'] if word_info['pos'] in ['PROPN']]
        for word in words:
            if word in male_names:
                # take the word in the male_names list and replace it with the most similar name
                matches = process.extract(word, male_names, limit=2)
                # Filtra il match con punteggio 100 (se esiste)
                matches = [match for match in matches if match[1] < 100]
                if matches:
                    best_match = matches[0]
                    print("Nome più simile:", best_match[0])
                    print("Punteggio:", best_match[1])
                else:
                    best_match = word
                replacement_dict[word] = best_match[0]
            elif word in female_names:
                matches = process.extract(word, female_names, limit=2)
                # Filtra il match con punteggio 100 (se esiste)
                matches = [match for match in matches if match[1] < 100]
                if matches:
                    best_match = matches[0]
                    print("Nome più simile:", best_match[0])
                    print("Punteggio:", best_match[1])
                else:
                    best_match = word
                replacement_dict[word] = best_match[0]
            elif word in cities:
                matches = process.extract(word, cities, limit=2)
                # Filtra il match con punteggio 100 (se esiste)
                matches = [match for match in matches if match[1] < 100]
                if matches:
                    best_match = matches[0]
                    print("Nome più simile:", best_match[0])
                    print("Punteggio:", best_match[1])
                else:
                    best_match = word
                replacement_dict[word] = best_match[0]
    return replacement_dict

replacement = generate_replacement_dict()
print(replacement)

['Aamir', 'Aaron', 'Abbey', 'Abbie', 'Abbot', 'Abbott', 'Abby', 'Abdel', 'Abdul', 'Abdulkarim']
['Abagael', 'Abagail', 'Abbe', 'Abbey', 'Abbi', 'Abbie', 'Abby', 'Abigael', 'Abigail', 'Abigale']
['les Escaldes', 'Andorra la Vella', 'Umm Al Quwain City', 'Ras Al Khaimah City', 'Zayed City', 'Khawr Fakkān', 'Dubai', 'Dibba Al-Fujairah', 'Dibba Al-Hisn', 'Sharjah']
Nome più simile: Romain
Punteggio: 91
Nome più simile: Bryant
Punteggio: 90
Nome più simile: Elena
Punteggio: 91
Nome più simile: Elena
Punteggio: 91
Nome più simile: Elena
Punteggio: 91
Nome più simile: Elena
Punteggio: 91
Nome più simile: Libon
Punteggio: 91
Nome più simile: Laç
Punteggio: 90
Nome più simile: Ada
Punteggio: 90
Nome più simile: Sophi
Punteggio: 91
Nome più simile: Er
Punteggio: 90
Nome più simile: Mitchael
Punteggio: 93
Nome più simile: Mitchael
Punteggio: 93
Nome più simile: Ian
Punteggio: 90
Nome più simile: Son
Punteggio: 90
Nome più simile: Richardo
Punteggio: 93
Nome più simile: Harrison
Punteggio: 86
Nome

In [19]:
# FIFTH TECHNIQUE: Replace a proper noun with a similar proper noun

def replace_proper_noun(sample):
    sentence = sample.copy()
    words = [word_info['text'] for word_info in sentence['wsd']['hypothesis'] if word_info['pos'] in ['PROPN']]
    print("Words:", words)
    for word in words:
        if word in replacement:
            sentence['hypothesis'] = sentence['hypothesis'].replace(word, replacement[word])
            if sentence['label'] == 0:
                sentence['label'] = 2
            elif sentence['label'] == 2:
                sentence['label'] = 0
            elif sentence['label'] == 1:
                sentence['label'] = 1
    return sentence


In [20]:
# NOW LET'S APPLY RANDOMLY THE FIVE TECHNIQUES TO 15.000 RANDOM SAMPLES OF THE TRAINING DATASET

# Create a list of functions: each function corresponds to a technique
techniques = [
    replace_with_hypernym_hyponym, 
    apply_swap, 
    replace_with_antonym, 
    paraphrase, 
    replace_proper_noun]

# From the training dataset, select a random sample and repeat the process, creating a new dataset with 15000 samples.
# The original training set is stored in the variable 'train_dataset'. We generate new samples and then we add them to the original training set.

print("Number of samples in the original training dataset:", len(train_dataset))
print(train_dataset)

train_samples = [train_dataset[i] for i in range(len(train_dataset))]
print("Number of samples in the train_samples list:", len(train_samples))
print("Example sample:", train_samples[0])

# Randomly select 15000 samples
samples = random.sample(train_samples, 15000)
print("Number of samples:", len(samples))
print("Example sample:", samples[0])


Number of samples in the original training dataset: 51086
Dataset({
    features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
    num_rows: 51086
})
Number of samples in the train_samples list: 51086
Example sample: {'id': '150448', 'premise': "Roman Atwood . He is best known for his vlogs , where he posts updates about his life on a daily basis . His vlogging channel , `` RomanAtwoodVlogs '' , has a total of 3.3 billion views and 11.9 million subscribers . He also has another YouTube channel called `` RomanAtwood '' , where he posts pranks .", 'hypothesis': 'Roman Atwood is a content creator.', 'label': 0, 'wsd': {'premise': [{'index': 0, 'text': 'Roman', 'pos': 'ADJ', 'lemma': 'roman', 'bnSynsetId': 'bn:00109913a', 'wnSynsetOffset': '2921569a', 'nltkSynset': 'roman.a.01'}, {'index': 1, 'text': 'Atwood', 'pos': 'PROPN', 'lemma': 'Atwood', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 2, 'text': '.', 'pos': 'PUNCT', 'lemma': '.', 'bnSynsetId': 'O',

In [28]:
first_example = samples[0]
words = [word_info['text'] for word_info in first_example['wsd']['hypothesis'] if word_info['pos'] in ['NOUN']]
words_to_replace = random.sample(words, min(1, len(words)))
for word in words_to_replace:
    new_sentence = replace_with_hypernym_hyponym(first_example, word)
    print(new_sentence['hypothesis'])

The lead role in the [MASK] was played by Dennis Hopper .
tensor([6])
The lead role in the film was played by Dennis Hopper .


In [33]:

new_samples = []
print(samples[0]['hypothesis'])
def apply_technique(new_samples, samples, techniques):
    for sample in samples:
        function_applied = False
        print("Original sample:", sample['hypothesis'])
        
        while not function_applied:
            # Choose a random technique
            function = random.choice(techniques)
            print("Function to apply:", function.__name__)
            
            if function == apply_swap:
                #print("Applying apply_swap...")
                swapped_samples = apply_swap(sample)
                new_samples.extend(swapped_samples)
                function_applied = True
            elif function == replace_with_antonym:
                #print("Applying replace_with_antonym...")
                new_sentence = replace_with_antonym(sample)
                new_samples.append(new_sentence)
                function_applied = True
            elif function == paraphrase:
                #print("Applying paraphrase...")
                paraphrased_sentences = paraphrase(sample, num_return_sequences=3)
                best_paraphrase = check_paraphrases(sample, paraphrased_sentences)
                sample['hypothesis'] = best_paraphrase
                new_samples.append(sample)
                function_applied = True
            elif function == replace_proper_noun:
                #print("Applying replace_proper_noun...")
                new_sentence = replace_proper_noun(sample)
                #print("New sentence:", new_sentence['hypothesis'])
                new_samples.append(new_sentence)
                function_applied = True
            else:
                #print("Applying replace_with_hypernym_hyponym...")
                words = [word_info['text'] for word_info in sample['wsd']['hypothesis'] if word_info['pos'] in ['NOUN']]
                print("Words:", words)
                words_to_replace = random.sample(words, min(1, len(words)))
                if not words_to_replace:
                    print("No words to replace, choosing a different technique.")
                    continue  # Skip to the next iteration and choose a different technique
                for word in words_to_replace:
                    new_sentence = replace_with_hypernym_hyponym(sample, word)
                    #print("New sentence:", new_sentence['hypothesis'])
                new_samples.append(new_sentence)
                function_applied = True
    return new_samples

# Apply the techniques to the samples
new_samples = apply_technique(new_samples, samples, techniques)
print("Number of new samples:", len(new_samples))


The lead role in the show was played by Dennis Hopper.
Original sample: The lead role in the show was played by Dennis Hopper.
Function to apply: replace_with_antonym
Words: ['lead']
Comparatives: []
Antonym of lead: []
Original sample: The Girl on the Train is a mystery thriller film.
Function to apply: apply_swap
Original sample: James Arness was born in 2012.
Function to apply: paraphrase


c:\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Similarity scores: [0.24159130454063416, 0.18515963852405548, 0.173208087682724]
Original sample: Adrien Broner was ranked fifth best boxer in the world by The Ring magazine.
Function to apply: replace_with_hypernym_hyponym
Words: ['boxer', 'world', 'magazine']
Original sample: Shakira is only a mechanic.
Function to apply: replace_with_hypernym_hyponym
Words: ['mechanic']
Shakira is only a [MASK] .
tensor([7])
Original sample: Jeffrey Dahmer was convicted of a crime.
Function to apply: apply_swap
Original sample: Slipknot is a lawyer.
Function to apply: replace_with_hypernym_hyponym
Words: ['lawyer']
Slipknot is a [MASK] .
tensor([6])
Original sample: play the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the [unused0]
Function to apply: replace_proper_noun
Words: ['Maisie', 'Williams', 'Arya', 'Stark']
Original s

c:\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Similarity scores: [0.7284295558929443, 0.7284295558929443, 0.7284295558929443]
Original sample: Vera Farmiga was an orphan.
Function to apply: replace_with_antonym
Words: []
Original sample: Avatar required advanced technology.
Function to apply: replace_proper_noun
Words: ['Avatar']
Original sample: The Lion King includes the character Mufasa.
Function to apply: replace_with_hypernym_hyponym
Words: ['character']
The Lion King includes the [MASK] Mufasa .
tensor([6])
Original sample: Bette Davis is an astronomical object.
Function to apply: replace_with_hypernym_hyponym
Words: ['object']
Bette Davis is an astronomical [MASK] .
tensor([7])
Original sample: Cricket has players.
Function to apply: apply_swap
Original sample: Barbara Bain acts in television.
Function to apply: replace_proper_noun
Words: ['Barbara', 'Bain']
Original sample: The criticism of Destiny was mainly derived from the game's storyline and post-campaign content.
Function to apply: replace_proper_noun
Words: ['Destin

...

Similarity scores: [0.25672268867492676, 0.25380009412765503, 0.25898805260658264]
Original sample: Hayden Panetierre played the lead role, Juliette Barnes, on Nashville.
Function to apply: replace_proper_noun
Words: ['Hayden', 'Panetierre', 'Juliette', 'Barnes', 'Nashville']
Original sample: Luxembourg is one of the smallest sovereign states in Asia.
Function to apply: replace_proper_noun
Words: ['Luxembourg', 'Asia']
Original sample: Titanic had record-setting sales.
Function to apply: replace_with_hypernym_hyponym
Words: ['record', 'sales']
Titanic had record-setting [MASK] .
tensor([6])
Original sample: Ray Milland worked with Paramount film studios.
Function to apply: paraphrase


c:\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Similarity scores: [0.1835021823644638, 0.18319052457809448, 0.18345609307289124]
Original sample: Borussia Dortmund is a football club in Germany.
Function to apply: apply_swap
Original sample: Christopher Columbus encountered the Indies.
Function to apply: apply_swap
Original sample: Linux distributes tombs.
Function to apply: apply_swap
Original sample: Tony Blair is a leader.
Function to apply: replace_with_hypernym_hyponym
Words: ['leader']
Tony Blair is a [MASK] .
tensor([5])
Original sample: Melisandre appeared in A Storm of Swords (2000).
Function to apply: replace_with_antonym
Words: []
Original sample: Dracula is a novel written by someone who died in 1912.
Function to apply: replace_with_antonym
Words: []
Original sample: Beastie Boys was never formed.
Function to apply: replace_proper_noun
Words: ['Beastie', 'Boys']
Original sample: Alandi is in the Indian state of Maharashtra.
Function to apply: replace_with_hypernym_hyponym
Words: ['state']
Alandi is in the Indian [MASK] 

c:\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Similarity scores: [0.017590269446372986, -0.026483692228794098, -0.00992860272526741]
Original sample: Alicia Keys has been the winner of the NAACP Image Award 17 times.
Function to apply: apply_swap
Original sample: The Boston Celtics play at TD Garden for their home games.
Function to apply: paraphrase
Similarity scores: [0.10990922152996063, 0.05017803609371185, 0.0849839523434639]
Original sample: Robert Browning was a poet.
Function to apply: replace_with_antonym
Words: []
Original sample: Yale School of Drama provided training in only acting.
Function to apply: replace_with_antonym
Words: ['only']
Comparatives: []
Antonym of only: []
Original sample: Floyd Mayweather Jr. was born in February.
Function to apply: replace_with_hypernym_hyponym
Words: []
No words to replace, choosing a different technique.
Function to apply: replace_proper_noun
Words: ['Floyd', 'Mayweather', 'Jr.', 'February']
Original sample: Corey Taylor joined Slipknot in 1997.
Function to apply: apply_swap
Origi

In [64]:
# Create a new dataset with the new samples
new_samples_df = pd.DataFrame(new_samples)

new_samples_dataset = Dataset.from_pandas(new_samples_df)
print("New samples dataset created")

new_samples_dataset = new_samples_dataset.remove_columns(['wsd', 'srl'])
#train_dataset = train_dataset.remove_columns(['wsd', 'srl'])
#val_dataset = val_dataset.remove_columns(['wsd', 'srl'])

new_train_dataset = new_samples_dataset.select(range(10000))
new_val_dataset = new_samples_dataset.select(range(10000, 15000))

print(new_samples_dataset)
print(train_dataset)


# Concatenate the original training dataset with the new dataset
training_dataset = datasets.concatenate_datasets([train_dataset, new_train_dataset])
validation_dataset = datasets.concatenate_datasets([val_dataset, new_val_dataset])
print("New training dataset created")
print("New validation dataset created")
print(training_dataset)
print(validation_dataset)


New samples dataset created
Dataset({
    features: ['id', 'premise', 'hypothesis', 'label'],
    num_rows: 15000
})
Dataset({
    features: ['id', 'premise', 'hypothesis', 'label'],
    num_rows: 51086
})
New training dataset created
New validation dataset created
Dataset({
    features: ['id', 'premise', 'hypothesis', 'label'],
    num_rows: 61086
})
Dataset({
    features: ['id', 'premise', 'hypothesis', 'label'],
    num_rows: 7288
})


In [65]:
# Now save the new training dataset.
new_dataset = DatasetDict({
    'train': training_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

print("New dataset created")
print(new_dataset)

New dataset created
DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label'],
        num_rows: 61086
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label'],
        num_rows: 7288
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2287
    })
})


In [66]:
new_dataset.save_to_disk('dataset_dict')

Saving the dataset (1/1 shards): 100%|██████████| 2287/2287 [00:00<00:00, 15321.88 examples/s]


In [67]:
dataset_loaded = DatasetDict.load_from_disk('dataset_dict')
print(dataset_loaded)

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label'],
        num_rows: 61086
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label'],
        num_rows: 7288
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2287
    })
})
